In [1]:
!pip install -q tensorflow pillow numpy ipywidgets requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.0 MB/s eta 0:00:00


In [4]:
import os
import io
import requests
import numpy as np
import tensorflow as tf
from PIL import Image as PILImage
import ipywidgets as widgets
from IPython.display import display, Image

def run_otitis_demo_tflite(
    model_url: str = "https://raw.githubusercontent.com/averksuu/resume/main/projects/otitis-detection/models/otitis_cnn_best.tflite",
    model_path: str = "models/otitis_cnn_best.tflite"
):

    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    if not os.path.exists(model_path):
        print("Downloading TFLite model...")
        r = requests.get(model_url)
        r.raise_for_status()
        with open(model_path, "wb") as f:
            f.write(r.content)
        print("Model saved to", model_path)
    else:
        print("Model already exists at", model_path)


    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    in_details  = interpreter.get_input_details()
    out_details = interpreter.get_output_details()
    print("Interpreter loaded. Input shape:", in_details[0]['shape'])

    def preprocess_image_from_bytes(image_bytes):
        img = PILImage.open(io.BytesIO(image_bytes)).convert('RGB').resize((128,128))
        arr = np.array(img, dtype=np.float32) / 255.0
        return arr


    uploader = widgets.FileUpload(accept='image/*', multiple=False)
    output   = widgets.Output()

    def on_upload_change(change):
        output.clear_output()
        for _, file_info in uploader.value.items():
            img_bytes = file_info['content']
            arr = preprocess_image_from_bytes(img_bytes)[np.newaxis, ...]
            interpreter.set_tensor(in_details[0]['index'], arr)
            interpreter.invoke()
            prob = interpreter.get_tensor(out_details[0]['index'])[0][0]
            label = 'Otitis' if prob >= 0.5 else 'Normal'

            with output:
                display(Image(data=img_bytes))
                print(f"Probability of Otitis: {prob:.4f}")
                print(f"Predicted class      : {label}")

    uploader.observe(on_upload_change, names='value')
    display(widgets.VBox([
        widgets.Label("Upload an ear image (.jpg/.png):"),
        uploader,
        output
    ]))


run_otitis_demo_tflite()

Model saved to models/otitis_cnn_best.tflite
Interpreter loaded. Input shape: [  1 128 128   3]
